In [3]:
%run prophet_funcs.py

In [64]:
import pandas as pd
import time
from datetime import timedelta

#get the time now in unix time
now = int(time.time())

#get the time one year from now datetime object
day_lag = datetime.today() - timedelta(days = 120 )

#convert datetime object to unix timestamp
then = int(day_lag.timestamp())

url = {
    'SOL': F'https://query1.finance.yahoo.com/v7/finance/download/SOL1-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true',
    'ETH': F'https://query1.finance.yahoo.com/v7/finance/download/ETH-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true',
    'XRP': F'https://query1.finance.yahoo.com/v7/finance/download/XRP-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true',
    'BTC': F'https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true'
}




In [77]:


def fetchData(url):
    for i in url.items():
        df = pd.DataFrame()
        key = i[0]
        val = i[1]
        data = pd.read_csv(val)
        data = data.assign(token = key) 
        df = df.append(data)
        return(df)


df = fetchData(url)

In [72]:
for x in url.items():
    key = x[0]
    val = x[1]
    print('key',key,'val',val)

key SOL val https://query1.finance.yahoo.com/v7/finance/download/SOL1-USD?period1=1624246284&period2=1634614284&interval=1d&events=history&includeAdjustedClose=true
key ETH val https://query1.finance.yahoo.com/v7/finance/download/ETH-USD?period1=1624246284&period2=1634614284&interval=1d&events=history&includeAdjustedClose=true
key XRP val https://query1.finance.yahoo.com/v7/finance/download/XRP-USD?period1=1624246284&period2=1634614284&interval=1d&events=history&includeAdjustedClose=true
key BTC val https://query1.finance.yahoo.com/v7/finance/download/BTC-USD?period1=1624246284&period2=1634614284&interval=1d&events=history&includeAdjustedClose=true


In [62]:
df = pd.DataFrame()

df = df.append(file)

In [59]:
import pandas as pd
import time
from datetime import timedelta

#get the time now in unix time
now = int(time.time())

#get the time one year from now datetime object
day_lag = datetime.today() - timedelta(days = 30 )

#convert datetime object to unix timestamp
then = int(day_lag.timestamp())

#Build Yahoo Finance Url request for SOL one year worth of data from Now 
#url = F'https://query1.finance.yahoo.com/v7/finance/download/SOL1-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true' #SOL
url = F'https://query1.finance.yahoo.com/v7/finance/download/ETH-USD?period1={then}&period2={now}&interval=1d&events=history&includeAdjustedClose=true'

file2 = pd.read_csv(url)

In [69]:
del(df)